In [66]:
import sys
import glob

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import netCDF4 as nc

In [147]:
PATH = './data/pr_24hr_50yrs/'

In [148]:
nc_list = np.asarray(glob.glob(PATH+"*"))

In [149]:
dataset = nc.Dataset(nc_list[0])

In [150]:
dataset_list = np.asarray([nc.Dataset(path) for path in nc_list])

In [151]:
dataset_list.shape, nc_list.shape

((35,), (35,))

In [152]:
# Choose one random instance
# -------------------------- #
indx = np.random.randint(0, dataset_list.shape[0])
inst = dataset_list[indx]

In [153]:
inst['pr']

<class 'netCDF4._netCDF4.Variable'>
float64 pr(rlat, rlon)
    units: mm h-1
    _FillValue: nan
    long_name: 50-year 24-hour precipitation flux
unlimited dimensions: 
current shape = (124, 149)
filling on

In [154]:
inst['lat']

<class 'netCDF4._netCDF4.Variable'>
float64 lat(rlat, rlon)
    units: degrees_north
    _FillValue: nan
    long_name: latitude
unlimited dimensions: 
current shape = (124, 149)
filling on

In [155]:
rlat = inst.variables['rlat'][:]
rlon = inst.variables['rlon'][:]
print('rlat', rlat.shape, 'rlon', rlon.shape)

rlat (124,) rlon (149,)


In [175]:
lat = [full.variables['lat'][:, :] for full in dataset_list]

lon = [full.variables['lon'][:, :] for full in dataset_list]
pr = [full.variables['pr'][:, :] for full in dataset_list]

pr={}
for full in dataset_list:
    pr[full] = full.variables['pr'][:, :]
    
df_pr = pd.Panel(pr)

CanRCM4 = (df_pr, rlat, rlon)

(<class 'pandas.core.panel.Panel'>
 Dimensions: 35 (items) x 124 (major_axis) x 149 (minor_axis)
 Items axis: <class 'netCDF4._netCDF4.Dataset'>
 root group (NETCDF4 data model, file format HDF5):
     Author: Chao Li @ PCIC - chaoli@uvic.ca
     Institution: Pacific Climate Impacts Consortium
     Date created: Tue Aug 28 00:10:49 2018
     dimensions(sizes): rlon(149), rlat(124)
     variables(dimensions): float64 rlon(rlon), float64 rlat(rlat), float64 lon(rlat,rlon), float64 lat(rlat,rlon), float64 pr(rlat,rlon)
     groups: 
  to <class 'netCDF4._netCDF4.Dataset'>
 root group (NETCDF4 data model, file format HDF5):
     Author: Chao Li @ PCIC - chaoli@uvic.ca
     Institution: Pacific Climate Impacts Consortium
     Date created: Tue Aug 28 00:13:45 2018
     dimensions(sizes): rlon(149), rlat(124)
     variables(dimensions): float64 rlon(rlon), float64 rlat(rlat), float64 lon(rlat,rlon), float64 lat(rlat,rlon), float64 pr(rlat,rlon)
     groups: 
 
 Major_axis axis: 0 to 123
 Min

In [157]:
rlat_indx = range(0, rlat.shape[0])
rlong_indx = range(0, rlon.shape[0])

In [124]:
df = pd.DataFrame()
df['rlat'] = rlat

df.append(df_lat)

TypeError: cannot concatenate object of type "<class 'numpy.ma.core.MaskedArray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [52]:
x

<class 'netCDF4._netCDF4.Variable'>
float64 rlat(rlat)
    units: degrees
    long_name: latitude in rotated pole grid
unlimited dimensions: 
current shape = (124,)
filling on, default _FillValue of 9.969209968386869e+36 used